In [1]:
import pandas as pd
import os
import numpy as np
import shutil

### Enter the path to weather_station full_dataset

In [2]:
weather_data_file_path= r'C:\Users\raramachandr\OneDrive - NVIDIA Corporation\Documents\Personal\Masters_files_desktop\project\New_files\smokeynet-engineering-main1\data\raw\weather_raw'

### Import the camera_to_weather_station mapping file

In [3]:
camera_station_mapping = pd.read_csv("../../data/processed/camera_station_mappings.csv")
#camera_station_mapping.head()

In [4]:
camera_station_mapping.shape

(309, 7)

### Import the camera ID to camera_name file (provided in Fiblib_files)

In [5]:
camera_station_mapping2 = pd.read_csv("../../data/raw/camera_id_mapping.csv")
#camera_station_mapping2.head()

### Merge both the mappnig files

In [6]:
camera_station_mapping3 = pd.merge(camera_station_mapping,camera_station_mapping2, how='left', left_on ="properties.description.id", right_on = "Camera_ID")

### Create weighted average question

In [7]:
# Find max distance and subract from the distance column (to get inverse of distance)
max_distance_calculated = camera_station_mapping3.distance_mi.max()+1
camera_station_mapping3["distance_mi_inverse"] = max_distance_calculated - camera_station_mapping3["distance_mi"]
#camera_station_mapping3.head()

### Calculate the weighted factor and save to mapping file (not needed)

first =1
list_of_unique_camera = list(camera_station_mapping3.camera_name.unique())
for interate in range(len(list_of_unique_camera)):
    camera_name = list_of_unique_camera[interate]
    one_camera_at_time = camera_station_mapping3[camera_station_mapping3.camera_name == camera_name].copy()
    distance_sum = one_camera_at_time.distance_mi_inverse.sum()
    one_camera_at_time["weight_factor"] = one_camera_at_time.distance_mi_inverse/distance_sum
    if (first):
        camera_station_mapping4 = one_camera_at_time.copy()
        first =0
    else:
        camera_station_mapping4 = pd.concat([camera_station_mapping4,one_camera_at_time])

In [8]:
#camera_station_mapping4 = camera_station_mapping3

### Split the weather data into multiple small chuck of file.

In [9]:
class FileSettings(object):
    def __init__(self, file_name, row_size=1000000):
        self.file_name = file_name
        self.row_size = row_size


class FileSplitter(object):

    def __init__(self, file_settings):
        self.file_settings = file_settings

        if type(self.file_settings).__name__ != "FileSettings":
            raise Exception("Please pass correct instance ")

        self.df = pd.read_csv(self.file_settings.file_name,
                              chunksize=self.file_settings.row_size)

    def run(self, directory="split_files"):
        try:os.makedirs(directory)
        except Exception as e:pass

        counter = 0

        while True:
            try:
                file_name = "{}/{}_{}_row_{}.csv".format(
                    directory,  self.file_settings.file_name.split(".")[0], counter, self.file_settings.row_size
                ) 
                df = next(self.df).to_csv(file_name,index =False)
                counter = counter + 1
            except StopIteration:
                break
            except Exception as e:
                print("Error:",e)
                break

        return True

os.chdir(weather_data_file_path)

helper =  FileSplitter(FileSettings(
    file_name='weather_HPWREN.csv',
    row_size=2000000
))
helper.run()

In [10]:
### Run the filter spliter for the 3 group of weather Stations
os.chdir(weather_data_file_path)

helper =  FileSplitter(FileSettings(
    file_name='weather_HPWREN.csv',
    row_size=2000000
))
helper.run()

helper =  FileSplitter(FileSettings(
    file_name='weather_SC-EDISON.csv',
    row_size=2000000
))
helper.run()

helper =  FileSplitter(FileSettings(
    file_name='weather_SDGE.csv',
    row_size=2000000
))
helper.run()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


True

### Create the folder inside the temp directory weather_data by weatherstation directory

In [11]:
weather_data_file_path_split = weather_data_file_path + r'\split_files'
os.chdir(weather_data_file_path_split)
shutil.rmtree(weather_data_file_path_split +r'\ByWeatherStation',ignore_errors=True)

weather_split_file_list = os.listdir(weather_data_file_path_split)
try:os.makedirs("ByWeatherStation")
except Exception as e:pass

In [12]:
os.getcwd()

'C:\\Users\\raramachandr\\OneDrive - NVIDIA Corporation\\Documents\\Personal\\Masters_files_desktop\\project\\New_files\\smokeynet-engineering-main1\\data\\raw\\weather_raw\\split_files'

### split the weather data by station

In [13]:
def split_weather_station_by_file (file_weather_data):
    file_chunk_number = int(file_weather_data.split('_row_')[0].split("_")[-1])
    if(file_chunk_number == 0):
        temp_file1_df = pd.read_csv(file_weather_data,header=[0,1])
        temp_file1_df.columns = temp_file1_df.columns.droplevel(1)
    else:
        temp_file1_df = pd.read_csv(file_weather_data)
    
    weather_stid_in_temp_file = temp_file1_df.Station_ID.unique()
    for j in range(len(weather_stid_in_temp_file)):
        temp_weather_stid = weather_stid_in_temp_file[j]
        temp_file_iter = temp_file1_df[temp_file1_df.Station_ID == temp_weather_stid]
        if (temp_weather_stid+".csv" in os.listdir(r"ByWeatherStation/")):
            file_1 = pd.read_csv(r"ByWeatherStation/"+temp_weather_stid+".csv")
            file_1 = pd.concat([file_1,temp_file_iter])
            file_1.to_csv(r"ByWeatherStation/"+temp_weather_stid+".csv",index = False)
        else:
            temp_file_iter.to_csv(r"ByWeatherStation/"+temp_weather_stid+".csv", index = False)

In [14]:
for i in range(len(weather_split_file_list)):
    file_weather_data = weather_split_file_list [i]
    split_weather_station_by_file(file_weather_data)

In [15]:
file_weather_data

'weather_SDGE_9_row_2000000.csv'

## Weather_data calculation by camera name

In [16]:
camera_name_unique_list = camera_station_mapping3.camera_name.unique()
len(camera_station_mapping3.camera_name.unique())

127

### create weather station folder by camera

In [23]:
camera_data_path = weather_data_file_path + r'\by_camera_summary'
os.chdir(weather_data_file_path)
shutil.rmtree(camera_data_path,ignore_errors=True)
os.mkdir("by_camera_summary")

os.chdir(weather_data_file_path + r"\split_files\ByWeatherStation")

In [24]:
###  remove the station id which donot have any weather_data

In [25]:
list_weather_station_data_csv = os.listdir()
weather_station_data_available = []
for i in range (len(list_weather_station_data_csv)):
    temp1= list_weather_station_data_csv[i].split(".")[0]
    weather_station_data_available.append(temp1)

In [26]:
camera_station_mapping3.shape

(381, 10)

In [27]:
camera_station_mapping4 = camera_station_mapping3[camera_station_mapping3['stid'].isin(weather_station_data_available)]
#print(store_df_filtered)

In [28]:
camera_station_mapping4.shape

(381, 10)

In [29]:
camera_station_mapping4.head()

,properties.description.id,stid,shortname,distance_m,distance_mi,is_in_direction,rn,Camera_ID,camera_name,distance_mi_inverse
0,hpwren0_unknown direction,BFDSD,SDGE,2362.963219,1.468277,NaN,1,hpwren0_unknown direction,tje-1-mobo-c,25.642502
1,hpwren0_unknown direction,CVXSD,SDGE,8161.908209,5.071575,NaN,2,hpwren0_unknown direction,tje-1-mobo-c,22.039205
2,hpwren0_unknown direction,DJZSD,SDGE,10216.081526,6.347979,NaN,3,hpwren0_unknown direction,tje-1-mobo-c,20.762800
3,hpwren1_north,HP016,HPWREN,0.000000,0.000000,True,1,hpwren1_north,bm-n-mobo-c,27.110779
4,hpwren1_north,HP016,HPWREN,0.000000,0.000000,True,1,hpwren1_north,bm-n-mobo,27.110779


### Loop through all weather stations to calculate the per camera summary

In [30]:
for i in range (len(camera_name_unique_list)):
    # Get the weather station related to camera
    camera_name = camera_name_unique_list[i]
    weather_station_of_intrest = list(camera_station_mapping4[camera_station_mapping4["camera_name"] == camera_name ].stid.unique())
    
    #read all the weather station data into single dataframe:
    #print(os.listdir())
    for j in range (len(weather_station_of_intrest)):    
        temp_df = pd.read_csv(weather_station_of_intrest[j] +".csv")
        if (j == 0):
            weather_station_of_intrest_df = temp_df.copy()
        else:
            weather_station_of_intrest_df =  pd.concat([weather_station_of_intrest_df,temp_df])
            
    # Stack all the weather data into single 
    weather_station_of_intrest_df_stacked = pd.melt(weather_station_of_intrest_df, id_vars=['Station_ID', 'Date_Time'],
        var_name='weather_parameter_name', value_name='weather_parameter_value')
    
    # extract key columns from the camera_station mapping file
    camera_station_mapping5 = camera_station_mapping4[camera_station_mapping4.camera_name == camera_name]
    camera_station_mapping5 = camera_station_mapping5 [["camera_name","distance_mi_inverse", "stid"]]
    
    # join the weather data to mapping file.
    weather_station_of_intrest_df_stacked = pd.merge(weather_station_of_intrest_df_stacked, camera_station_mapping5, left_on= "Station_ID", right_on= "stid")
  
    #aggregate the columns to find the sum of all the distance per time/weather_parameter.
    weather_station_of_intrest_df_pivot = pd.pivot_table (weather_station_of_intrest_df_stacked, values="distance_mi_inverse" , 
                                                index= ['Date_Time', 'weather_parameter_name'],
                                                 aggfunc='sum')
    weather_station_of_intrest_df_pivot.reset_index(inplace=True)
    weather_station_of_intrest_df_pivot.rename(columns = {'distance_mi_inverse': 'distance_mi_inverse_sum'}, inplace =True)
    
    
    #Join the weighted sum of distance to stacked columns and calculate the weight factor and normalize the weather data. 
    weather_station_of_intrest_df_stacked_with_weight = pd.merge(weather_station_of_intrest_df_stacked,weather_station_of_intrest_df_pivot, on= ['Date_Time', 'weather_parameter_name'])
    weather_station_of_intrest_df_stacked_with_weight['weather_parameter_value_norm'] =  (
    weather_station_of_intrest_df_stacked_with_weight['weather_parameter_value'] * weather_station_of_intrest_df_stacked_with_weight['distance_mi_inverse'] 
    / weather_station_of_intrest_df_stacked_with_weight['distance_mi_inverse_sum'])
    
    #Round data to nearest 3 digit. (run time is high)
    decimals = 4
    weather_station_of_intrest_df_stacked_with_weight['weather_parameter_value_norm'] = weather_station_of_intrest_df_stacked_with_weight['weather_parameter_value_norm'].apply(lambda x: round(x, decimals))
    
    # Calculate the weighted weather parameter value by per camera + Time. 
    weather_data_per_camera_agg = pd.pivot_table (weather_station_of_intrest_df_stacked_with_weight, values="weather_parameter_value_norm" , 
                                                index= ['Date_Time', 'camera_name'], columns= 'weather_parameter_name',
                                                 aggfunc='sum')
    weather_data_per_camera_agg.reset_index(inplace=True)
    
    weather_data_per_camera_agg_path = camera_data_path + "\\" + camera_name + r'.csv'
    weather_data_per_camera_agg.to_csv(weather_data_per_camera_agg_path,index=False)
    #print (i)                

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126


### Some debug steps

In [ ]:
available_station_in_mapping = set(camera_station_mapping4.weather_file_name.unique())

In [ ]:
camera_station_mapping4['weather_file_name'] = camera_station_mapping4.stid + ".csv"

camera_station_mapping4.head()

In [ ]:
os.chdir(weather_data_file_path + r"\split_files\ByWeatherStation")
available_station_in_file = set(os.listdir())

In [ ]:
missing_file = list (available_station_in_mapping - available_station_in_file)

In [ ]:
missing_file 

camera_station_mapping4_missing_weather = camera_station_mapping4[camera_station_mapping4['weather_file_name'].isin(missing_file)]
camera_station_mapping4_missing_weather

camera_station_mapping4_missing_weather['properties.description.id'].unique()

In [ ]:
camera_station_mapping5 = camera_station_mapping3[~camera_station_mapping3['stid'].isin(weather_station_data_available)]


In [ ]:
camera_station_mapping5.head()

In [ ]:
camera_station_mapping5.stid.unique()